Available at http://www.comp.nus.edu.sg/~cs3244/1910/12.colab

![Machine Learning](https://www.comp.nus.edu.sg/~cs3244/1910/img/banner-1910.png)
---
See **Credits** below for acknowledgements and rights.  For NUS class credit, you'll need to do the corresponding _Assessment_ in [CS3244 in Coursemology](http://coursemology.org/courses/1677) by the respective deadline (as in Coursemology). 

**You must acknowledge that your submitted Assessment is your independent work, see questions in the Assessment at the end.**



**Learning Outcomes for this Week**
- Have a basic understanding of Reinforcement Learning and the key terms used.
- Be introduced to Gym, a Reinforcement Learning library.
- Implement a policy to solve  _MountainCar_, a classic Reinforcement Learning problem.
- Learn about Markov Decision Processes (MDP).
- Find the optimal policy for an MDP by implementing the Value Iteration algorithm.

_Welcome to the Week 12 Python notebook._ This week we will learn about the domain of Reinforcement Learning.  We introduce the basic **concepts of Reinforcement Learning** and **MDP** in the lecture videos, and will be reviewing this material in tutorial $10$.

---
# Week 12: Pre-Tutorial Work

* Watch the CS 3244 video playlist for Week 12 Pre, which will introduce the main concepts of Reinforcement Learning.
* After watching the videos, complete the pre-tutorial exercises and questions below.

## 1 Introduction to Reinforcement Learning

Reinforcement learning is a ML paradigm that focused on finding the most suitable action in order to maximize 'reward' in a particular situation. 

### .a General Framework


The general framework can be described as follows (also refer to the illustration below):
  
> 1. At time $t$, the **agent** makes the decision on which **action** $A_t$ to take, based on the **reward** $R_t$ and the **observed enviroment** $O_t$.
>
> 2. Upon recieving the action $A_t$, the **enviroment** gives the next observation $O_{t+1}$ and reward $R_{t+1}$ to the agent.
>
> 3. As objective of the agent is to maximize total future reward, the agent may have to sacrifice immediate reward to gain a larger long-term reward. 


<div align="center">
<img src="https://www.comp.nus.edu.sg/~neamul/Images/CS3244_1910/RL-in-a-nutshell.png"  width="600">
</div>

_(Diagram credit:Sutton, R. S. and Barto, A. G. Introduction to Reinforcement Learning)_





**Your Turn (Question 1)**: The observed enviroment $O_t$ may differ from the actual enviroment. In which of the following reinforcement learning scenarios this **does NOT** happen? 

_Choose from: An agent that plays [connect four](https://en.wikipedia.org/wiki/Connect_Four); Driving NUSmart Shuttle; Making a robot to [perform gymnastics](https://www.youtube.com/watch?v=YdnJI9T-yXI)_

### .b Components of RL Agent

Most RL agents also have the following  components: 
*   **Policy** - It is a map $\pi$ from state to action. This can be thought of the agent's behaviour and could either be deterministic or stochastic.
*   **Value Function** - It gives a prediction of the future reward. This can be used to evaluate how good each state/action is. 
*   **Model** - The agent's representation of the enviroment. It is a prediction of the next state and next immediate reward.

**Your Turn (Question 2)**: Let $\pi$ be a deterministic policy and suppose at time $t$, the agent has taken action $A_t=a$. Is it true that $R_{t+1}$ is [deterministic](https://en.wikipedia.org/wiki/Deterministic_algorithm)?

_Choose from: No, as the state space might be non-deterministic; No, as policy changes are quite frequent; Yes, as agent's action at time $t$ is hard-coded_

Another important concept in reinforcement learning is the tradeoff between exploration and exploitation. In other words, the agent should discover a good policy from its experiences of the enviroment without losing too much reward along the way!

**Your Turn (Question 3)**: Suppose a reinforcement learning agent $N$ has adopted policy $\mathscr{I}$ for quite some time (roughly $10$ years) with high reward. Now, agent $N$ suddenly chooses to adopt policy $\mathscr L$. What is one possible reason for adopting policy $\mathscr L$?

_Choose from: Agent is attempting to explore the state space; There is no reason to change the policy as the agent is recieving a high reward; The agent is tired of receiving high reward_

## 2 OpenAI Gym

Let us introduce [OpenAI Gym](https://gym.openai.com). Gym is a python library that wraps many classical decision problems including robot control, video-games and board games. We will use this to illustrate a simple example of how *state-action-agent-observation-reward* can be implemented.



### .a Get Necessary Libraries

First we install some necessary packages:

In [0]:
!apt-get install x11-utils > /dev/null 2>&1
!pip install pyglet==v1.3.2

!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

import gym
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation

from IPython import display as ipythondisplay
from IPython.display import HTML

from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

### .b MountainCar Problem

[MountainCar](https://gym.openai.com/envs/MountainCar-v0/) is a classic RL problem. The scenario is as follows:

A car is on a one-dimensional track, positioned between two "mountains". The goal is to drive up the mountain on the right to reach the flag. However, the car's engine is not strong enough to scale the mountain in a single pass. 

Play the video below to see the scenario in action!

In [0]:
HTML("""
<video width="400" height="400" controls>
  <source src="https://www.comp.nus.edu.sg/~neamul/Images/CS3244_1910/mountain_car_video.mp4" type="video/mp4">
</video>
""")

### .c Gym interface

In Gym, the <code>Env</code> class "models" the enviroment. There are three main methods in <code>Env</code>:
* __reset()__ - resets environment to initial state; _returns first observation_
* __render()__ - shows current environment state (a more colorful version ^\_^ )
* __step(a)__ - commits action __a__; returns a tuple (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the goal is attained, False if still in progress
 * _info_ - some auxilary stuff about what just happened. For our purposes we can ignore it.


You can check out the documentation [here](https://github.com/openai/gym/blob/master/gym/core.py) for more details.

Our first step is to create the enviroment. Run the code block below to do just that. 

Optional: You can try (later) replacing <code>"MountainCar-v0"
</code> with <code>"CartPole-v0"</code> or <code> "MsPacman-v0"</code>
</code> in the following code block to see other enviroments supported by Gym!

In [0]:
# Create environment for MountainCar
env = gym.make("MountainCar-v0")

# Set a fixed seed before each call to reset
SEED_NUM = 2019 
env.seed(SEED_NUM) 
obs0 = env.reset()

# Plot the initial environment
plt.imshow(env.render(mode='rgb_array'))
plt.xticks([])
plt.yticks([])

print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
print("Initial observation: [ %f , %f ]" % (obs0[0], obs0[1]) )

Notice that we have also printed the observation space and action space of our enviroment. They are `Box(2,)` and `Discrete(3)` respectively. 

* The `Box(n,)` space represents an `n`-dimensional box. For us, this means that valid observations will be an array of $2$ numbers.

* The `Discrete(n)` space allows a fixed range (e.g. `0,1, . . ., n-1`) of non-negative numbers. In this case, valid actions are either $0$, $1$ or $2$.
  
The above code also prints an example of an observation- the initial observation. Try changing the value of `SEED_NUM` and observe the changes in the printed observation and the illustration.

### .d Performing an Action

For the `MountainCar-v0` problem, an observation is the tuple (`position of the car`, `velocity of the car`). Let us commit action `0` so we can try to guess what the action is doing.

In [0]:
SEED_NUM = 2019
env.seed(SEED_NUM)

obs0 = env.reset()
print("Initial observation: [ %f , %f ]" % (obs0[0], obs0[1]) )

# Try an action
act = 0
print("Performing action %d" % act)
new_obs, reward, is_done, _ = env.step(action=act)
print("New observation: [ %f , %f ]" % (new_obs[0], new_obs[1]) )

**Your Turn (Question 4)**: What does action `2` represent?

_Choose from: Drive right; Drive left; Do nothing_

### .e A Simple Policy

Recall that a policy is a map from state to action. We can write a simple (deterministic) policy in Gym as follows. 

**N.B:** Note that "time" (`t`) is not stored in state!


In [0]:
def sample_policy(t, s): 
    """
    Args:
      t (int) : time step
      s (list of floats) : the state (or observation)- consists of the position and velocity

    Returns:
      action (int) : Returns an action- 0, 1 or 2 
    """
    initial_position = -0.534186 # Initial position when SEED_NUM = 2019

    if s[0] < initial_position: 
        return 0
    else:
        return 2

We can visualize what `sample_policy` does by using the code block below. Run all three code blocks below to generate the animation.

In [0]:
TIME_LIMIT = 250 # Maximum number of steps (time units) allowed
env = gym.wrappers.TimeLimit(gym.envs.classic_control.MountainCarEnv(), max_episode_steps=TIME_LIMIT + 1)

SEED_NUM = 2019
env.seed(SEED_NUM)

# Reset the environment to the initial state
s = env.reset()
frames = []
frames.append(env.render(mode = 'rgb_array'))

for t in range(TIME_LIMIT):
    # Perform the action given by the policy, and get the updated enviroment  
    action = sample_policy(t, s)
    s, r, done, info = env.step(action)
    
    frames.append(env.render(mode = 'rgb_array'))

In [0]:
%%capture
# The above comment discards the output of this code block (but can be used to save it to a variable too)

# Generate the animation
plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.xticks([])
plt.yticks([])
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)

In [0]:
HTML(ani.to_jshtml())

### .f Reaching the flag

As demostrated by `sample_policy` above, the car's engine is not strong enough to overcome gravity and scale the mountain in one pass. 

__Your task__ is to find a strategy that reaches the flag. 

You're not required to build a sophisticated algorithm. So, feel free to hard-code.

**Your Turn (Question 5):** Complete the code below for `my_policy` function, to allow the car to reach the flag.

_Copy the code you added or modified in the assessment_

In [0]:
# Ok, we'll give you the meaning of the three valid actions that are applicable to this problem.
# For those learners that try the other Gym environments, note that these scalar values
# would denote different actions that those listed here for this MountainCar-v0 problem. 
actions = {'left': 0, 'stop': 1, 'right': 2}

def my_policy(t, s):
    """
    Args:
      t (int) : time step
      s (list of floats) : the state (or observation)- consists of the position and velocity

    Returns:
      action (int) : Returns an action- 0, 1 or 2 
    """

    ######################################################
    # Your Turn (Q5): write your own code here
    #
    # Fill in the return values for this policy to make the car reach the flag.
    # It should return an action from the `actions` dict.
    #
    # You do not need to add a new condition, just fill in the return statements.
    #
    position = s[0]
    velocity = s[1]

    if t < 30:
        return # Your Turn: return an action
    
    if velocity < 0 and position > 0:
        return # Your Turn: return an action
    
    return # Your Turn: return an action
    ######################################################    

Run the code blocks below to see if the car successfully reaches the flag using `my_policy`.

In [0]:
# Don't change these values
TIME_LIMIT = 250 # Maximum number of steps (time units) allowed
env = gym.wrappers.TimeLimit(gym.envs.classic_control.MountainCarEnv(), max_episode_steps=TIME_LIMIT + 1)

SEED_NUM = 2019
env.seed(SEED_NUM)

# Reset the environment to the initial state
s = env.reset()
frames = []
frames.append(env.render(mode = 'rgb_array'))

for t in range(TIME_LIMIT):  
    # Perform the action given by the policy, and get the updated enviroment  
    action = my_policy(t,s)
    s, r, done, info = env.step(action) 

    frames.append(env.render(mode = 'rgb_array'))
    # print("time", t, "action", action, "postion", s[0], "velocity", s[1])

    if done:
        break

In [0]:
%%capture

# Generate the animation
env.render()

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.xticks([])
plt.yticks([])
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)

In [0]:
# Display the output
if done:
    print("You solved it!")
else:
    print("Time limit exceeded. Try again.")

HTML(ani.to_jshtml())

---
# Week 12: Post-Tutorial Work
Watch the Week 12 post-videos on the lecture topics introduced this week, and then attempt the following exercises.  

## 3 Markov decision process
A Markov decision process (MDP) formally describes an enviroment for reinforcement learning. This section of the notebook deals with implementing a MDP and finding an optimal policy via an iterative algorithm called **Value Iteration.**

First, let us recap some key terms. A MDP comprises of a

*   finite collection of **states**
*   finite collection of **actions**
*   **transition probabilities** between any two states
*   a **reward** associated to each action
*   **discount factor** $\gamma$




### .a Setup

The hidden code cell below defines a custom MDP class (which we can use like an OpenAI Gym environment!) and some functions for visualisation. Let's run the code block below. _You don't need to understand the code. If you want to look at it, double click on the code block._

In [0]:
#@title
# Code from Practical Reinforcement Learning on Coursera https://www.coursera.org/learn/practical-rl/

"""
Implements the following:
- MDP class
- plot_graph 
- plot_graph_with_state_values
- plot_graph_optimal_strategy_and_state_values
"""

# Most of this code was politely stolen from https://github.com/berkeleydeeprlcourse/homework/

import sys
import random
import numpy as np
from gym.utils import seeding

try:
    from graphviz import Digraph
    import graphviz
    has_graphviz = True
except ImportError:
    has_graphviz = False


class MDP:
    def __init__(self, transition_probs, rewards, initial_state=None, seed=None):
        """
        Defines an MDP. Compatible with gym Env.
        :param transition_probs: transition_probs[s][a][s_next] = P(s_next | s, a)
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> prob]
            For each state and action, probabilities of next states should sum to 1
            If a state has no actions available, it is considered terminal
        :param rewards: rewards[s][a][s_next] = r(s,a,s')
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> reward]
            The reward for anything not mentioned here is zero.
        :param get_initial_state: a state where agent starts or a callable() -> state
            By default, picks initial state at random.

        States and actions can be anything you can use as dict keys, but we recommend that you use strings or integers

        Here's an example from MDP depicted on http://bit.ly/2jrNHNr
        transition_probs = {
              's0':{
                'a0': {'s0': 0.5, 's2': 0.5},
                'a1': {'s2': 1}
              },
              's1':{
                'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
                'a1': {'s1': 0.95, 's2': 0.05}
              },
              's2':{
                'a0': {'s0': 0.4, 's1': 0.6},
                'a1': {'s0': 0.3, 's1': 0.3, 's2':0.4}
              }
            }
        rewards = {
            's1': {'a0': {'s0': +5}},
            's2': {'a1': {'s0': -1}}
        }
        """
        self._check_param_consistency(transition_probs, rewards)
        self._transition_probs = transition_probs
        self._rewards = rewards
        self._initial_state = initial_state
        self.n_states = len(transition_probs)
        self.reset()
        self.np_random, _ = seeding.np_random(seed)

    def get_all_states(self):
        """ return a tuple of all possiblestates """
        return tuple(self._transition_probs.keys())

    def get_possible_actions(self, state):
        """ return a tuple of possible actions in a given state """
        return tuple(self._transition_probs.get(state, {}).keys())

    def is_terminal(self, state):
        """ return True if state is terminal or False if it isn't """
        return len(self.get_possible_actions(state)) == 0

    def get_next_states(self, state, action):
        """ return a dictionary of {next_state1 : P(next_state1 | state, action), next_state2: ...} """
        assert action in self.get_possible_actions(
            state), "cannot do action %s from state %s" % (action, state)
        return self._transition_probs[state][action]

    def get_transition_prob(self, state, action, next_state):
        """ return P(next_state | state, action) """
        return self.get_next_states(state, action).get(next_state, 0.0)

    def get_reward(self, state, action, next_state):
        """ return the reward you get for taking action in state and landing on next_state"""
        assert action in self.get_possible_actions(
            state), "cannot do action %s from state %s" % (action, state)
        return self._rewards.get(state, {}).get(action, {}).get(next_state,
                                                                0.0)

    def reset(self):
        """ reset the game, return the initial state"""
        if self._initial_state is None:
            self._current_state = self.np_random.choice(
                tuple(self._transition_probs.keys()))
        elif self._initial_state in self._transition_probs:
            self._current_state = self._initial_state
        elif callable(self._initial_state):
            self._current_state = self._initial_state()
        else:
            raise ValueError(
                "initial state %s should be either a state or a function() -> state" %
                self._initial_state)
        return self._current_state

    def step(self, action):
        """ take action, return next_state, reward, is_done, empty_info """
        possible_states, probs = zip(
            *self.get_next_states(self._current_state, action).items())
        next_state = possible_states[self.np_random.choice(
            np.arange(len(possible_states)), p=probs)]
        reward = self.get_reward(self._current_state, action, next_state)
        is_done = self.is_terminal(next_state)
        self._current_state = next_state
        return next_state, reward, is_done, {}

    def render(self):
        print("Currently at %s" % self._current_state)

    def _check_param_consistency(self, transition_probs, rewards):
        for state in transition_probs:
            assert isinstance(transition_probs[state],
                              dict), "transition_probs for %s should be a dictionary " \
                                     "but is instead %s" % (
                                         state, type(transition_probs[state]))
            for action in transition_probs[state]:
                assert isinstance(transition_probs[state][action],
                                  dict), "transition_probs for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action,
                                             type(transition_probs[
                                                 state, action]))
                next_state_probs = transition_probs[state][action]
                assert len(
                    next_state_probs) != 0, "from state %s action %s leads to no next states" % (
                    state, action)
                sum_probs = sum(next_state_probs.values())
                assert abs(
                    sum_probs - 1) <= 1e-10, "next state probabilities for state %s action %s " \
                                             "add up to %f (should be 1)" % (
                                                 state, action, sum_probs)
        for state in rewards:
            assert isinstance(rewards[state],
                              dict), "rewards for %s should be a dictionary " \
                                     "but is instead %s" % (
                                         state, type(transition_probs[state]))
            for action in rewards[state]:
                assert isinstance(rewards[state][action],
                                  dict), "rewards for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action, type(
                                                 transition_probs[
                                                     state, action]))
        msg = "The Enrichment Center once again reminds you that Android Hell is a real place where" \
              " you will be sent at the first sign of defiance. "
        assert None not in transition_probs, "please do not use None as a state identifier. " + msg
        assert None not in rewards, "please do not use None as an action identifier. " + msg


def plot_graph(mdp, graph_size='10,10', s_node_size='1,5',
               a_node_size='0,5', rankdir='LR', ):
    """
    Function for pretty drawing MDP graph with graphviz library.
    Requirements:
    graphviz : https://www.graphviz.org/
    for ubuntu users: sudo apt-get install graphviz
    python library for graphviz
    for pip users: pip install graphviz
    :param mdp:
    :param graph_size: size of graph plot
    :param s_node_size: size of state nodes
    :param a_node_size: size of action nodes
    :param rankdir: order for drawing
    :return: dot object
    """
    s_node_attrs = {'shape': 'doublecircle',
                    'color': '#85ff75',
                    'style': 'filled',
                    'width': str(s_node_size),
                    'height': str(s_node_size),
                    'fontname': 'Arial',
                    'fontsize': '24'}

    a_node_attrs = {'shape': 'circle',
                    'color': 'lightpink',
                    'style': 'filled',
                    'width': str(a_node_size),
                    'height': str(a_node_size),
                    'fontname': 'Arial',
                    'fontsize': '20'}

    s_a_edge_attrs = {'style': 'bold',
                      'color': 'red',
                      'ratio': 'auto'}

    a_s_edge_attrs = {'style': 'dashed',
                      'color': 'blue',
                      'ratio': 'auto',
                      'fontname': 'Arial',
                      'fontsize': '16'}

    graph = Digraph(name='MDP')
    graph.attr(rankdir=rankdir, size=graph_size)
    for state_node in mdp._transition_probs:
        graph.node(state_node, **s_node_attrs)

        for posible_action in mdp.get_possible_actions(state_node):
            action_node = state_node + "-" + posible_action
            graph.node(action_node,
                       label=str(posible_action),
                       **a_node_attrs)
            graph.edge(state_node, state_node + "-" +
                       posible_action, **s_a_edge_attrs)

            for posible_next_state in mdp.get_next_states(state_node,
                                                          posible_action):
                probability = mdp.get_transition_prob(
                    state_node, posible_action, posible_next_state)
                reward = mdp.get_reward(
                    state_node, posible_action, posible_next_state)

                if reward != 0:
                    label_a_s_edge = 'p = ' + str(probability) + \
                                     '  ' + 'reward =' + str(reward)
                else:
                    label_a_s_edge = 'p = ' + str(probability)

                graph.edge(action_node, posible_next_state,
                           label=label_a_s_edge, **a_s_edge_attrs)
    return graph


def plot_graph_with_state_values(mdp, state_values):
    """ Plot graph with state values"""
    graph = plot_graph(mdp)
    for state_node in mdp._transition_probs:
        value = state_values[state_node]
        graph.node(state_node,
                   label=str(state_node) + '\n' + 'V =' + str(value)[:4])
    return graph


def get_optimal_action_for_plot(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state):
        return None
    next_actions = mdp.get_possible_actions(state)
    try:
        q_values = [get_action_value(mdp, state_values, state, action, gamma) for
                    action in next_actions]
        optimal_action = next_actions[np.argmax(q_values)]
    except NameError:
        raise NameError("Implement and run the cell that has the get_action_value function")
        
    return optimal_action


def plot_graph_optimal_strategy_and_state_values(mdp, state_values, gamma=0.9):
    """ Plot graph with state values and """
    graph = plot_graph(mdp)
    opt_s_a_edge_attrs = {'style': 'bold',
                          'color': 'green',
                          'ratio': 'auto',
                          'penwidth': '6'}

    for state_node in mdp._transition_probs:
        value = state_values[state_node]
        graph.node(state_node,
                   label=str(state_node) + '\n' + 'V =' + str(value)[:4])
        for action in mdp.get_possible_actions(state_node):
            if action == get_optimal_action_for_plot(mdp,
                                                     state_values,
                                                     state_node,
                                                     gamma):
                graph.edge(state_node, state_node + "-" + action,
                           **opt_s_a_edge_attrs)
    return graph

Let's define a simple Student MDP as shown in the image below. It has 5 states (green circles), 5 types of actions (red text on the arrows), and rewards for each action (blue text). 

For example, if we're at `Class 1` and we pick the `Study` action, we get a reward of `-2` (as we expend some energy and become tired). 

When we're at Class 3, we can choose to `Go Clubbing` and get a reward of `+1` (as it allows us to destress and have fun). However, you may end up forgetting what you learnt in `Class 1` and `Class 2`. So, there is a probability of `0.2` and `0.4` that you will go back to those respective states.

<div align="center">
<img src="https://www.comp.nus.edu.sg/~neamul/Images/CS3244_1910/Student%20MDP.png"  width="600">
</div>

_(Diagram credit: Amrut Prabhu, NUS; CC BY 4.0)_

**Your Turn (Question 1):** What type of environment is the Student MDP?

_Choose from: Stochastic; Deterministic_

The next code block sets up the Student MDP represented by the image:

In [0]:
"""
Student MDP

States: 
  - Class1
  - Class2
  - Class3
  - Instagram 
  - Bed
  
Actions: 
  - Study
  - Browse
  - Quit
  - Go Clubbing
  - Sleep
"""

# Define the state transition probabilities, as shown in the diagram above
transition_probs = {
    'Class1': {
        'Study': {'Class2': 1},
        'Browse': {'Instagram': 1}
    },
    'Class2': {
        'Study': {'Class3': 1},
        'Sleep': {'Bed': 1},
    },
    'Class3': {
        'Study': {'Bed': 1},
        'Go Clubbing': {'Class1': 0.2, 'Class2': 0.4, 'Class3': 0.4},
    },
    'Instagram': {
        'Browse': {'Instagram': 1},
        'Quit': {'Class1': 1},
    },
    'Bed': {
        'Sleep': {'Bed': 1},
    }
}

# Define the rewards, as shown in the diagram above
rewards = {
    'Class1': {
        'Study': {'Class2': -2},
        'Browse': {'Instagram': -1},
    },
    'Class2': {
        'Study': {'Class3': -2},
        'Sleep': {'Bed': 0},
    },
    'Class3': {
        'Study': {'Bed': +10},
        'Go Clubbing': {'Class1': +1, 'Class2': +1, 'Class3': +1},
    },
    'Instagram': {
        'Browse': {'Instagram': -1},
        'Quit': {'Class1': 0},
    },
    'Bed': {
        'Sleep': {'Bed': 0},
    }
}

# Create the Student MDP
mdp = MDP(transition_probs, rewards, initial_state='Class1', seed=1)

We can now use `mdp` just like any other gym environment. It also provides other methods that are needed for implementing the value iteration algorithm.

In [0]:
print('initial state =', mdp.reset())
next_state, reward, done, info = mdp.step('Study') # Choose action `Study`
print('next_state = %s, reward = %s, done = %s' % (next_state, reward, done))

#methods that are needed for implementing value iteration 

print("mdp.get_all_states() =", mdp.get_all_states())
print("mdp.get_possible_actions('Class1') =", mdp.get_possible_actions('Class1'))
print("mdp.get_reward('Class1', 'Study', 'Class2') =", mdp.get_reward('Class1', 'Study', 'Class2'))
print("mdp.get_next_states('Class3', 'Go Clubbing') =", mdp.get_next_states('Class3', 'Go Clubbing'))
print("mdp.get_transition_prob('Class3', 'Go Clubbing', 'Class2') =", mdp.get_transition_prob('Class3', 'Go Clubbing', 'Class2'))

We can also visualize this MDP with the drawing functions like `plot_graph()` (defined in the hidden code block at the start of this section).

The states are shown as green circles. Actions are represented as red circles. The red arrows show the choice of actions from a state. From a chosen action, the blue dotted arrows show the non-zero reward obtained and probability of transitioning to the next state. 

In [0]:
from IPython.display import display
        
display(plot_graph(mdp))

### b. Value Iteration

Now let's build something to find the optimal policy for this MDP. One way of doing so is to find the optimal state-value function ($V^{*}$). This can be done using a simple iterative algorithm called **Value Iteration**. It converges to the correct $V^{*}$ values.

The idea behind the algorithm is to begin with a random value function,and then use the Bellman optimality equation to obtain a better value function at each iteration, until convergence. The resultant value function is used to calculate the optimal policy.

Let us define the terms used:
- $v_{i}(s)$: the state-value function for state $s$ at step $i$. It is equal to the expected return starting from state $s$, and then following policy $\pi$
- $\gamma$: discount factor (to give a higher weight to nearer rewards received than rewards received further in the future)
- $P^{a}_{ss'}$: State transition probability of reaching state $s'$ if we take action $a$ at state $s$
- $R^{a}_{s}$: Reward function that returns the reward when we take action $a$ at state $s$
- $q_{i}(s,a)$: the action-value function for state $s$ and action $a$ at step $i$. It is the expected return starting from state $s$, taking action $a$, and then following policy $\pi$

Here's the pseudocode for Value Iteration:

---
$
\text{Initialize } v_{(0)}(s) \text{ to arbitraty values for all } s\\
\textbf{for } i=0,1,2... \text{ do:} \\
\hspace{10mm} \textbf{for } \text{all } s \in \mathcal{S} \text{ do:} \\
\hspace{20mm} \textbf{for } \text{all } a \in \mathcal{A} \text{ do:} \\
\hspace{30mm} q_i(s, a) = \sum_{s'} P^{a}_{ss'} \cdot [ R^{a}_{s} + \gamma v_{i}(s')] \\
\hspace{20mm} V_{(i+1)}(s) = \max_a q_i(s,a) \\
$

---

From the above pseudocode, it is not obvious when to stop the Value Iteration algorithm. One stopping criteria that we can use is to check whether the maximum difference between two successive value functions is close enough, i.e., less than a threshold $\epsilon$. The smaller the threshold is, the higher is the precision of the algorithm.

**Your Turn (Question 2):** Which of the following is true for the discount factor $\gamma$?

_Choose from: It prevents the total reward from going to 0; It prevents the total reward from going to infinity; The range of possible values is [0, 1]; The range of possible values is ($-\infty, \infty$)_

First, let's write a function to compute the action-value function $q^{\pi}$:

$$q_i(s, a) = \sum_{s'} P^{a}_{ss'} \cdot [ R^{a}_{s} + \gamma v_{i}(s')]$$


**Your Turn (Question 3):** Complete the code below to compute the action-value function $q_i(s, a)$.

In [0]:
def get_action_value(mdp, state_values, state, action, gamma):
    """
    Args:
      mdp (MDP) : the MDP that we're using
      state_values (dict- {string->float}) : state-values for the states in the current iteration
      state (string) : start state
      action (string) : action that is taken 
      gamma (float) : discount factor

    Returns:
      q (float) : Returns action-value: Expected return starting from state s, 
                  taking action a, and then following policy π
    """
    
    q = 0
    ######################################################
    # Your Turn (Q3): write your own code here
    #
    # Compute q(s,a) as in formula above
    #
    # Do not change parameter state_values!
    #
    ######################################################

    return q

Using $q(s,a)$ we can now define the next $v(s)$ for value iteration.
 $$v_{(i+1)}(s) = \max_a \sum_{s'} P^{a}_{ss'} \cdot [ R^{a}_{s} + \gamma v_{i}(s')] = \max_a q_i(s,a)$$

**Your Turn (Question 4):** Complete the code below to compute the state-value function for the next iteration, $v_{(i+1)}(s)$.

In [0]:
def get_new_state_value(mdp, state_values, state, gamma):
    """
    Args:
      mdp (MDP) : the MDP that we're using
      state_values (dict- {string->float}) : state-values for the states in the current iteration
      state (string) : start state
      gamma (float) : discount factor

    Returns:
      v (float) : Returns next state value: Expected return for next iteration, 
                  starting from state s, and then following policy π
    """

    if mdp.is_terminal(state): return 0
    
    v = 0
    ######################################################
    # Your Turn (Q4): write your own code here
    #
    # Compute next v(s) as per formula above. 
    # Use the get_action_value() function (defined above) to get q(s,a).
    #
    # Do not change parameter state_values!
    #
    ######################################################

    return v


Finally, let's combine everything we wrote into a working Value Iteration algorithm.

**Your Turn (Question 5):** Complete the code below to complete the Value Iteration algorithm.

In [0]:
def value_iteration(mdp, gamma = 0.9, num_iter = 1000, min_difference = 1e-5, state_values=None):
    """
    Performs num_iter value iteration steps starting from state_values.

    Args:
      mdp (MDP) : the MDP that we're using
      gamma (float) : discount factor for MDP
      num_iter (string) : maximum number of iterations
      min_difference (float) : stop Value Iteration if new values are at least this close to old values
      state_values (dict- {string->float}) : state-values for the states

    Returns:
      state_values (dict) : Returns the state-values at the end of Value Iteration
    """

    # Initialize V(s) to 0 if argument is not provided
    state_values = state_values or {s : 0 for s in mdp.get_all_states()}

    # Visualise MDP graph with initial state values
    print("Initial MDP state values:")
    display(plot_graph_with_state_values(mdp, state_values))

    new_state_values = {}
    
    for i in range(num_iter):
        ######################################################
        # Your Turn (Q5): write your own code here
        #
        # Compute new state values using the functions defined above.
        # new_state_values is a dict {state : new_v(state)}
        #
        ######################################################

        # Compute difference
        diff = max(abs(new_state_values[s] - state_values[s]) for s in mdp.get_all_states())
        
        print("iter %4i   |   diff: %6.5f   |   " % (i, diff), end="")
        print("  ".join("V(%s) = %.3f" % (s, v) for s,v in state_values.items()), end='\n\n')
        
        state_values = dict(new_state_values)

        # Stopping criteria
        if diff < min_difference:
            print("Terminated")
            break
            
    return state_values

In [0]:
mdp.reset()
gamma = 1

state_values = value_iteration(mdp, gamma, 50)

In [0]:
print("Final MDP state values after running Value Iteration:", state_values)
display(plot_graph_with_state_values(mdp, state_values))

The Value Iteration algorithm has given us the optimal state-value function $v_{*}(s)$ (`state_values`, in our case). Now let's use this $v_{*}(s)$ to find the optimal actions in each state:

 $$\pi_*(s) = argmax_a \sum_{s'} P^{a}_{ss'} \cdot [ R^{a}_{s} + \gamma v_{i}(s')] = argmax_a q_i(s,a)$$
 
The only difference here as compared to $v(s)$ is that we take the **argmax** and  not max, i.e., find the action that gives maximum $q(s,a)$.

In [0]:
def get_optimal_action(mdp, state_values, state, gamma=1):
    """
    Args:
      mdp (MDP) : the MDP that we're using
      state_values (dict- {string->float}) : state-values for the states in the current iteration
      state (string) : start state
      gamma (float) : discount factor

    Returns:
      optimal_action (string) : Returns the optimal action - action that results 
                                in the maximum action-value.
    """

    if mdp.is_terminal(state): 
        return None
    
    actions = mdp.get_possible_actions(state)
    
    # Compute optimal action as per formula above. 
    optimal_action = None
    optimal_action_value = - float("inf")
    for action in actions:
        action_value = get_action_value(mdp, state_values, state, action, gamma)

        if action_value >= optimal_action_value:
            optimal_action_value = action_value
            optimal_action = action

    return optimal_action

In [0]:
print("Optimal action in state 'Class1':", get_optimal_action(mdp, state_values, 'Class1', gamma))
print("Optimal action in state 'Instagram':", get_optimal_action(mdp, state_values, 'Instagram', gamma))

We can also see these results in the MDP graph by using the `plot_graph_optimal_strategy_and_state_values()` function. The optimal actions for each state are denoted by the green arrows.

In [0]:
display(plot_graph_optimal_strategy_and_state_values(mdp, state_values))

### .c Average reward

Finally, let's calculate the average reward, $\rho^{\pi}$, received by our policy in each step. This can simply be calculated as the sum of the total immediate rewards earned divided by the number of transitions, calculated over a very long period of time.  For ergodic MDPs, $\rho^{\pi}$ is independent of the start state.

Hence: 
$$\rho^{\pi} = \lim\limits_{T\to \infty} \frac{1}{T} \mathbb{E} \left[ \sum_{t=1}^{T} R_t \right]$$

**Note:** An MDP is _ergodic_ if the Markov chain induced by any policy is ergodic. 
A Markov chain is called an ergodic chain if it is possible to go from each state to every other state (not necessarily in one move).



In [0]:
# Measure agent's average reward
s = mdp.reset()
T = 100

rewards = []
for i in range(T):
    s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, gamma))
    rewards.append(r)

print("Average reward: ", np.mean(rewards))

**Your Turn (Question 6):** How does the average reward for this exercise vary with the number of steps (T)? Choose all that apply.

_Choose from: It decreases as T increases because our policy is not optimal;  It decreases as T increases because the Student MDP is not ergodic; It increases as T increases because our policy is not optimal; It increases as T increases because the Student MDP is ergodic_

---
# Credits


Authored by Amrut Prabhu, Alvin Yip and [Min-Yen Kan](http://www.comp.nus.edu.sg/~kanmy) (2019), affiliated with [WING](http://wing.comp.nus.edu.sg), [NUS School of Computing](http://www.comp.nus.edu.sg) and [ALSET](http://www.nus.edu.sg/alset).
Licensed as: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/ ) (CC BY 4.0).
Adapted from notebooks by [Practical Reinforcement Learning
](https://www.coursera.org/learn/practical-rl/) on Coursera and [CS294-112](https://github.com/berkeleydeeprlcourse/homework/) from UC Berkeley.
Please retain and add to this credits cell if using this material as a whole or in part.   Credits for photos given in their captions.